# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Google

 - Needs a Google API key and module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, folium, Flask)


In [32]:
import time
from pprint import pprint
from ipywidgets import widgets, interact
import pdb

import numpy as np
import pandas as pd
import pandas_dedupe

from sklearn.preprocessing import StandardScaler

import requests, json 

import gmaps
with open('apikeys/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
from yelpapi import YelpAPI
with open('apikeys/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

from foursquare import Foursquare, FoursquareException
with open('apikeys/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('apikeys/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

gmaps.configure(api_key=api_key)

import folium


## Google Maps

In [3]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:',
    disabled=False,
)):
    global keyword
    keyword = kw
    return kw


interactive(children=(Dropdown(description='Search term:', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')),…

In [4]:
# pick a location
location = '40.7484, -73.9857'
location_coords = eval(location)
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.624468,-74.0487134'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = eval(loc)
    return loc

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [5]:
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [6]:
# global options for all search services APIs
MIN_USER_RATINGS = 40
MIN_RATING = 3
NRESULTS = 50
RADIUS = 3000


In [7]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [15]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + val
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location, keyword):

    # use either rankby or radius
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, rankby=rankby)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        return gmaps_df


In [19]:
# use either rankby or radius
gmaps_df = gmaps_get_df(location, keyword)
# runquery(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df


> <ipython-input-15-151dad72369a>(29)gmaps_get_all_df()
-> j = gmaps_get_first_page(api_key, location, **kwargs)
(Pdb) c


,name,address,rating,nratings,lat,lng
0,Pizza Moto,338 Hamilton Ave,4.7,278,40.674622,-74.000800
1,Juliana's,19 Old Fulton St,4.6,3961,40.702747,-73.993435
2,Sottocasa,298 Atlantic Ave,4.6,673,40.688307,-73.988978
3,Dellarocco's,214 Hicks St,4.6,431,40.695009,-73.996108
4,The House of Pizza & Calzone,132 Union St,4.6,316,40.683997,-74.002281
5,Piz-zetta Pizzeria,90 Livingston St,4.6,237,40.691169,-73.990632
6,&pizza - Wall Street,63 Wall St,4.6,113,40.705765,-74.008547
7,Lucali,575 Henry St,4.5,1292,40.681805,-74.000293
8,Patsy’s Pizzeria,450 Dean St,4.5,593,40.681828,-73.976196
9,Table 87,87 Atlantic Ave,4.5,476,40.691282,-73.997321


In [20]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [21]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## Yelp

In [40]:
location

'40.6915812,-73.9954095'

In [41]:
def yelp_get_df(location, keyword):
    lat, lng = eval(location)
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url
0,Juliana's Pizza,19 Old Fulton St,4.5,2200,40.702615,-73.993416,https://www.yelp.com/biz/julianas-pizza-brookl...
1,Lucali,575 Henry St,4.5,1562,40.681800,-74.000240,https://www.yelp.com/biz/lucali-brooklyn-3?adj...
2,Sottocasa Pizzeria,298 Atlantic Ave,4.5,629,40.688285,-73.989006,https://www.yelp.com/biz/sottocasa-pizzeria-br...
3,Piz-zetta,90 Livingston St,4.5,288,40.691283,-73.990603,https://www.yelp.com/biz/piz-zetta-brooklyn-2?...
4,Forcella Fried Pizza,445 Albee square W,4.5,45,40.690880,-73.982868,https://www.yelp.com/biz/forcella-fried-pizza-...
5,Forno Rosso,327 Gold St,4.0,682,40.694467,-73.982930,https://www.yelp.com/biz/forno-rosso-brooklyn?...
6,Front Street Pizza,80 Front St,4.0,359,40.702440,-73.989430,https://www.yelp.com/biz/front-street-pizza-br...
7,La Cigogne,215 Union St,4.0,356,40.683501,-73.999304,https://www.yelp.com/biz/la-cigogne-brooklyn?a...
8,Circa Brewing,141 Lawrence St,4.0,317,40.691650,-73.986030,https://www.yelp.com/biz/circa-brewing-brookly...
9,Table 87 - Brooklyn Heights,87 Atlantic Ave,4.0,272,40.691219,-73.997345,https://www.yelp.com/biz/table-87-brooklyn-hei...


In [42]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [43]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Foursquare

In [47]:
def foursquare_get_df(location, keyword):

    client = foursquare.Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        venue_url = venue['delivery.url']
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        venue_rating = -1
        venue_nratings = -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            # sometimes no rating ... probably not popular enough
            print(type(e), str(e))
            print(traceback.format_exc())
            print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    if len(foursquare_df) and len(foursquare_df.columns):
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        return foursquare_df
    else:
        return None

foursquare_df = foursquare_get_df(location, keyword)
foursquare_df

ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2742f558a46c5ff1efc090'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2742655016ad244152ca43'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2745bb6cf9db736bc8e351'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2744e8f598007f30a23a4b'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2745bb6cf9db736bc8e351'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27452ad79d8e3962ee11fa'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27459dccfebd3b5f1da8f1'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27452e589b4c32e7017b4a'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27459dccfebd3b5f1da8f1'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27453c2119245df4f06aed'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2744fdc36f8b4e4be274cd'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27453d2119245df4f06f23'}


...

ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27436e58a46c5ff1f143df'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2745b2f025076ada622815'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2744ffa12e764cfddc206c'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f2745b2f025076ada622815'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274537f773f578f1348014'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27440fd736fc78bc1a140e'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f27440fd736fc78bc1a140e'}


,name,address,rating,nratings,lat,lng,url
0,MontyQ's Brick Oven Pizza,158 Montague St,-1,-1,40.694370,-73.992879,https://www.seamless.com/menu/monty-qs-158-mon...
1,Juliana's Pizza,19 Old Fulton St,-1,-1,40.702769,-73.993616,NaN
2,Ignazio's Pizza,4 Water St,-1,-1,40.703299,-73.994029,https://www.seamless.com/menu/ignazios-pizza-4...
3,Fascati Pizza,80 Henry St,-1,-1,40.698355,-73.992522,NaN
4,Front Street Pizza,80 Front St,-1,-1,40.702523,-73.989640,https://www.seamless.com/menu/front-street-piz...
5,Jay St. Fresh 99¢ Pizza,408 Jay St,-1,-1,40.691787,-73.987462,NaN
6,99 Cent Pizza,51D Willoughby St,-1,-1,40.692165,-73.986972,https://www.seamless.com/menu/99-cent-supreme-...
7,2 Bros. Pizza,395 Flatbush Ave,-1,-1,40.689394,-73.981125,NaN
8,Mario's Pizza & Chicken,222 Hoyt St,-1,-1,40.683542,-73.990013,NaN
9,Giardini Pizza,363 Smith St,-1,-1,40.679354,-73.995358,https://www.seamless.com/menu/giardini-gourmet...


In [48]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [49]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Deduplicate and merge

In [50]:
pd.set_option('display.max_rows', None)

gmaps_df_copy = gmaps_df.copy()
gmaps_df_copy['source'] = '0_gmaps'

yelp_df_copy = yelp_df.copy() 
yelp_df_copy['source'] = '1_yelp'

foursquare_df_copy = foursquare_df.copy()
foursquare_df_copy['source'] = '2_foursquare'

venues_df = pd.concat([gmaps_df_copy, yelp_df_copy, foursquare_df_copy]).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,source,url,latlong
6,6,&pizza - Wall Street,63 Wall St,4.6,113,40.705765,-74.008547,0_gmaps,NaN,"(40.7057647, -74.0085474)"
87,34,2 Bros Pizza,395 Flatbush Ave,3.0,84,40.689339,-73.980896,1_yelp,https://www.yelp.com/biz/2-bros-pizza-brooklyn...,"(40.689338684082, -73.9808959960938)"
96,7,2 Bros. Pizza,395 Flatbush Ave,-1.0,-1,40.689394,-73.981125,2_foursquare,NaN,"(40.68939371833308, -73.98112529498947)"
95,6,99 Cent Pizza,51D Willoughby St,-1.0,-1,40.692165,-73.986972,2_foursquare,https://www.seamless.com/menu/99-cent-supreme-...,"(40.692165329405405, -73.98697185450945)"
99,10,99 Cent Pizza,255 Livingston St,-1.0,-1,40.688831,-73.983299,2_foursquare,https://www.seamless.com/menu/99-cents-hot-piz...,"(40.6888313293457, -73.9832992553711)"
44,44,99 Cents Hot Pizza,255 Livingston St,4.0,119,40.688824,-73.983285,0_gmaps,NaN,"(40.68882420000001, -73.98328459999999)"
138,49,AKA 99¢ Fresh Pizza,117 Orchard St,-1.0,-1,40.719254,-73.989562,2_foursquare,NaN,"(40.7192539686804, -73.98956199368286)"
102,13,Adrienne's Pizza Bar,54 Stone St,-1.0,-1,40.704348,-74.010223,2_foursquare,NaN,"(40.704348091685645, -74.01022261314714)"
15,15,Adrienne's Pizzabar,54 Stone St,4.4,1630,40.704294,-74.010065,0_gmaps,NaN,"(40.7042941, -74.0100646)"
47,47,Antonio's Pizzeria,32 Court St,3.9,109,40.692952,-73.991008,0_gmaps,NaN,"(40.6929516, -73.99100829999999)"


In [51]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 97


In [52]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'source']]
venues_df

,cluster,name,address,rating,nratings,lat,lng,source
1,0,Pizza Moto,338 Hamilton Ave,4.7,278,40.6746224,-74.00079989999999,0_gmaps
134,0,Pizza Moto,338 Hamilton Ave,-1.0,-1,40.67480235603758,-74.00037496011366,2_foursquare
2,1,Juliana's,19 Old Fulton St,4.6,3961,40.7027467,-73.9934349,0_gmaps
53,1,Juliana's Pizza,19 Old Fulton St,4.5,2200,40.7026153030093,-73.9934159993549,1_yelp
90,1,Juliana's Pizza,19 Old Fulton St,-1.0,-1,40.702769,-73.993616,2_foursquare
3,2,Sottocasa,298 Atlantic Ave,4.6,673,40.6883065,-73.9889778,0_gmaps
55,2,Sottocasa Pizzeria,298 Atlantic Ave,4.5,629,40.688285,-73.989006,1_yelp
4,3,Dellarocco's,214 Hicks St,4.6,431,40.6950091,-73.9961082,0_gmaps
63,3,Dellarocco's,214 Hicks St,4.0,267,40.6950309,-73.9961252,1_yelp
5,4,The House of Pizza & Calzone,132 Union St,4.6,316,40.6839972,-74.00228129999999,0_gmaps


In [53]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'source']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,source
0,0,Pizza Moto,338 Hamilton Ave,40.6746224,-74.00079989999999,0_gmaps
1,1,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,0_gmaps
2,2,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0_gmaps
3,3,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0_gmaps
4,4,The House of Pizza & Calzone,132 Union St,40.6839972,-74.00228129999999,0_gmaps
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,0_gmaps
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0_gmaps
7,7,Pizza Town,85 5th Ave,40.68,-73.9777778,0_gmaps
8,8,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0_gmaps
9,9,Brado,155 Atlantic Ave,40.69064520000001,-73.9949106,0_gmaps


In [54]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [55]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [57]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_rating_std,yelp_rating,yelp_rating_std,foursquare_rating,foursquare_rating_std
0,0,Pizza Moto,338 Hamilton Ave,40.6746224,-74.00079989999999,0_gmaps,4.7,1.538126,NaN,NaN,-1.0,0.0
1,1,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,0_gmaps,4.6,1.183688,4.5,1.535002,-1.0,0.0
2,2,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0_gmaps,4.6,1.183688,4.5,1.535002,NaN,NaN
3,3,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0_gmaps,4.6,1.183688,4.0,0.407245,NaN,NaN
4,4,The House of Pizza & Calzone,132 Union St,40.6839972,-74.00228129999999,0_gmaps,4.6,1.183688,4.0,0.407245,NaN,NaN
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,0_gmaps,4.5,0.829251,4.5,1.535002,NaN,NaN
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0_gmaps,4.5,0.829251,4.0,0.407245,NaN,NaN
7,7,Pizza Town,85 5th Ave,40.68,-73.9777778,0_gmaps,4.5,0.829251,NaN,NaN,-1.0,0.0
8,8,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0_gmaps,4.5,0.829251,4.0,0.407245,NaN,NaN
9,9,Brado,155 Atlantic Ave,40.69064520000001,-73.9949106,0_gmaps,4.5,0.829251,4.0,0.407245,NaN,NaN


In [58]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating
34,L'Arte Della Pizza Brooklyn,172 5th Ave,4.8,NaN,NaN
56,Piz-zetta,90 Livingston St,NaN,4.5,NaN
2,Sottocasa,298 Atlantic Ave,4.6,4.5,NaN
35,&pizza - Wall Street,63 Wall St,4.6,NaN,NaN
5,Lucali,575 Henry St,4.5,4.5,NaN
15,Forcella Fried Pizza,445 Albee Square W,4.4,4.5,NaN
1,Juliana's,19 Old Fulton St,4.6,4.5,-1.0
37,Union Street Pizza,226 4th Ave,4.5,NaN,NaN
36,Patsy’s Pizzeria,450 Dean St,4.5,NaN,NaN
3,Dellarocco's,214 Hicks St,4.6,4.0,NaN


In [59]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']]




mean number of ratings 1.4329896907216495
average rating 3.6421704980508733e-16


,name,address,gmaps_rating,yelp_rating,foursquare_rating,nratings,bayes_score
2,Sottocasa,298 Atlantic Ave,4.6,4.5,NaN,2,7.919309e-01
34,L'Arte Della Pizza Brooklyn,172 5th Ave,4.8,NaN,NaN,1,7.778758e-01
5,Lucali,575 Henry St,4.5,4.5,NaN,2,6.886862e-01
56,Piz-zetta,90 Livingston St,NaN,4.5,NaN,1,6.309119e-01
1,Juliana's,19 Old Fulton St,4.6,4.5,-1.0,3,6.132860e-01
15,Forcella Fried Pizza,445 Albee Square W,4.4,4.5,NaN,2,5.854416e-01
35,&pizza - Wall Street,63 Wall St,4.6,NaN,NaN,1,4.865160e-01
3,Dellarocco's,214 Hicks St,4.6,4.0,NaN,2,4.634252e-01
4,The House of Pizza & Calzone,132 Union St,4.6,4.0,NaN,2,4.634252e-01
0,Pizza Moto,338 Hamilton Ave,4.7,NaN,-1.0,2,4.480427e-01


In [90]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'source']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    merge_df[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']]
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe(dedupe_list)

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 96


,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_rating_std,yelp_rating,yelp_rating_std,foursquare_rating,foursquare_rating_std,nratings,w,R,bayes_score
2,2,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0,4.6,1.265430,4.5,1.535002,NaN,NaN,2,0.578313,1.400216,8.097635e-01
5,5,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0,4.6,1.265430,4.5,1.535002,NaN,NaN,2,0.578313,1.400216,8.097635e-01
6,6,Lucali,575 Henry St,40.6818053,-74.0002926,0,4.5,0.905781,4.5,1.535002,NaN,NaN,2,0.578313,1.220392,7.057688e-01
1,1,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,0,4.6,1.265430,4.5,1.535002,-1.0,0.0,3,0.672897,0.933477,6.281343e-01
16,16,Forcella Fried Pizza,445 Albee Square W,40.6911058,-73.9831758,0,4.4,0.546133,4.5,1.535002,NaN,NaN,2,0.578313,1.040568,6.017740e-01
36,36,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0,4.6,1.265430,NaN,NaN,NaN,NaN,1,0.406780,1.265430,5.147512e-01
3,3,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0,4.6,1.265430,4.0,0.407245,NaN,NaN,2,0.578313,0.836338,4.836652e-01
4,4,The House of Pizza & Calzone,132 Union St,40.6839972,-74.00228129999999,0,4.6,1.265430,4.0,0.407245,NaN,NaN,2,0.578313,0.836338,4.836652e-01
0,0,Pizza Moto,338 Hamilton Ave,40.6746224,-74.00079989999999,0,4.7,1.625078,NaN,NaN,-1.0,0.0,2,0.578313,0.812539,4.699022e-01
7,7,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0,4.5,0.905781,4.0,0.407245,NaN,NaN,2,0.578313,0.656513,3.796704e-01


In [91]:
# app.py

location = '40.6915812,-73.9954095'
keyword = 'pizza'
ltype = 'establishment'
rankby = 'distance'

gmaps_df = gmaps_get_df(location, keyword)
yelp_df = yelp_get_df(location, keyword)
foursquare_df = foursquare_get_df(location, keyword)
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))
print(len(gmaps_df), len(yelp_df), len(foursquare_df))
print("Deduping %d dataframes" % (len(list(dedupe_list))))
dedupe(dedupe_list).to_json()


> <ipython-input-15-151dad72369a>(29)gmaps_get_all_df()
-> j = gmaps_get_first_page(api_key, location, **kwargs)
(Pdb) c


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d1762c02d6bfb0aad8b'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d19ba97fe39f198f309'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d1962c02d6bfb0ab4c8'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d0c2119245df408af0b'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d1962c02d6bfb0ab4c8'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d5e8f44ac7ed564a102'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274ccec36f8b4e4bfac070'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274d41932b2b7420b04b5f'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274ccec36f8b4e4bfac070'}


...

ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274dd38f44ac7ed56602c4'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274ef57455323d64e9fd78'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274ce12378a6144b6cfbe2'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274ef57455323d64e9fd78'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274cfb8b9c0f54263bc67f'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274c0a0070593f9d23eae6'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f274c0a0070593f9d23eae6'}
53 36 50
Deduping 3 dataframes
Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 96


'{"cluster":{"2":2,"34":34,"5":5,"55":55,"1":1,"15":15,"35":35,"3":3,"4":4,"0":0,"9":9,"6":6,"8":8,"37":37,"36":36,"10":10,"7":7,"12":12,"39":39,"38":38,"58":58,"56":56,"57":57,"59":59,"61":61,"60":60,"16":16,"11":11,"17":17,"32":32,"40":40,"47":47,"46":46,"45":45,"44":44,"43":43,"42":42,"41":41,"21":21,"19":19,"78":78,"70":70,"71":71,"72":72,"77":77,"76":76,"75":75,"73":73,"74":74,"80":80,"79":79,"88":88,"94":94,"93":93,"92":92,"91":91,"90":90,"89":89,"87":87,"68":68,"86":86,"85":85,"84":84,"83":83,"82":82,"81":81,"69":69,"95":95,"67":67,"66":66,"65":65,"64":64,"13":13,"48":48,"28":28,"33":33,"20":20,"50":50,"49":49,"26":26,"27":27,"18":18,"62":62,"63":63,"24":24,"51":51,"14":14,"23":23,"22":22,"29":29,"25":25,"31":31,"53":53,"52":52,"54":54,"30":30},"name":{"2":"Sottocasa","34":"L\'Arte Della Pizza Brooklyn","5":"Lucali","55":"Piz-zetta","1":"Juliana\'s","15":"Forcella Fried Pizza","35":"&pizza - Wall Street","3":"Dellarocco\'s","4":"The House of Pizza & Calzone","0":"Pizza Moto","9"